In [2]:
import pickle

from tqdm import tqdm
import os.path
import pandas as pd
import numpy as np

# note 先利用pandas读取csv数据，过滤掉没什么用的ID
# 需要注意的是，SME的Code是2
cere_base_dir = "/data/hanzhi/CER-E/"
allocation_file = os.path.join(cere_base_dir, "SME and Residential allocations.csv")
df = pd.read_csv(allocation_file)
ids = df.get("ID")
codes = df.get("Code")
sme_ids = []
for i, c in zip(ids, codes):
    if c == 2:
        sme_ids.append(i)
file_names = [f"File{i}.txt" for i in range(1, 7)]
sme_ids = set(sme_ids)

# note 依次读取files，过滤掉不属于上面的id的内容，得到{id: [(timestamp, value)]}
sme_id2read = {i: [] for i in sme_ids}
timestamps = set()
for f in file_names:
    print(f"reading {f}")
    file = open(os.path.join(cere_base_dir, f))
    for line in file.readlines():
        items = line.split(" ")
        meter_id = int(items[0])
        timestamp = int(items[1])
        value = float(items[2])
        timestamps.add(timestamp)
        if meter_id not in sme_ids:
            continue
        # print(meter_id, timestamp, value)
        sme_id2read[meter_id].append((timestamp, value))

# note 再对上面的时间步进行排序，得到各个时间序列，到这一步应该有485条时间序列
min_ts = min(timestamps)
real_min_ts = (min_ts // 100) * 48 + (min_ts % 48 - 1)
print("min ts", min_ts, real_min_ts)

# note 目标文件
target_dir = "/data3/hanzhi/BasicTS/datasets/CER-E"
os.makedirs(target_dir, exist_ok=True)

# note 保存sme_id
# 先对sme_id进行排序
sme_ids = list(sme_ids)
sme_ids.sort()
np.savez(os.path.join(target_dir, "sme_ids.npz"), sme_ids=sme_ids)
# note 保存timestamp
timestamps = [real_min_ts + i for i in range(25728)]
np.savez(os.path.join(target_dir, "timestamps.npz"), timestamps=timestamps)

# 再保存
# note 保存原始时间序列，称为data_raw.dat，只有一个元素，没有filling
# 每一个都截断到只有前25728个
# 目标数组
arr = np.full(shape=(len(sme_id2read), 25728), fill_value=0., dtype=np.float32)
truncated = 0

for idx, sme_id in tqdm(enumerate(sme_ids)):
    sme_id2read[sme_id].sort()
    for ts, v in sme_id2read[sme_id]:
        # 一个三位数，到2009年1月1日的距离
        day_since_start = ts // 100
        # 一个两位数，0~47，表明今天的第几个半小时
        hour_of_day = ts % 100 -1
        real_ts = day_since_start * 48 + hour_of_day
        if real_ts - real_min_ts >= 25728:
            truncated += 1
        else:
            arr[idx, real_ts - real_min_ts] = v
print(f"truncated {truncated}", f"null value {np.sum(arr[arr==0.0])}")
np.savez(os.path.join(target_dir, "arr.npz"), arr=arr)
# note 将这些时间序列保存下来，暂时先不考虑空值的问题

reading File1.txt
reading File2.txt
reading File3.txt
reading File4.txt
reading File5.txt
reading File6.txt
min ts 19501 9372


485it [00:06, 75.03it/s]


truncated 0 null value 0.0


In [3]:
# note 先把arr增加day_of_week和time_of_day
target_dir = "/data3/hanzhi/BasicTS/datasets/CER-E"
original_data = np.load(os.path.join(target_dir, "arr.npz"))['arr']
timestamps = np.load(os.path.join(target_dir, "timestamps.npz"))['timestamps']
print("original_data_shape", original_data.shape, original_data.dtype, original_data[0,:48])
count_nodes, count_timestamps = original_data.shape
print("original_timestamp_shape", timestamps.shape)
dow = 2
tod = 0
dows = []
tods = []
for t in timestamps.tolist():
    dows.append(dow)
    tods.append(tod)
    tod += 1
    tod %= 48
    if tod == 0:
        dow += 1
        dow %= 7
# (count_timestamps) -> (1, count_timestamps) -> (count_nodes, timestamps)
dows = np.tile(np.expand_dims(np.array(dows, dtype=np.float32) / 7, axis=0), (count_nodes, 1))
tods = np.tile(np.expand_dims(np.array(tods, dtype=np.float32) / 48, axis=0), (count_nodes, 1))
# 3*(count_nodes, count_timestamps) -> (count_nodes, count_timestamps, 3)
new_data = np.stack((original_data, tods, dows), axis=-1)
new_data = new_data.transpose((1, 0, 2))
print("new data shape", new_data.shape)
written_data = np.memmap(os.path.join(target_dir, "data.dat"), mode="write", shape=new_data.shape, dtype=new_data.dtype)
print(new_data[:48, 0, :])
new_data[new_data == -32.0] = 0.
written_data[:] = new_data
written_data.flush()
print(np.sum(new_data[new_data == 0.0]))
print(np.sum(new_data == 0.))

original_data_shape (485, 25728) float32 [0.016 0.016 0.009 0.008 0.072 0.138 0.704 0.539 0.698 0.757 0.671 0.683
 0.676 0.553 0.757 0.768 0.588 0.633 0.999 0.879 0.813 0.681 0.63  0.778
 0.755 0.6   0.719 0.74  1.154 0.957 0.804 0.032 0.018 0.018 0.018 0.018
 0.018 0.017 0.017 0.017 0.017 0.018 0.018 0.018 0.018 0.018 0.018 0.018]
original_timestamp_shape (25728,)
new data shape (25728, 485, 3)
[[0.016      0.         0.2857143 ]
 [0.016      0.02083333 0.2857143 ]
 [0.009      0.04166667 0.2857143 ]
 [0.008      0.0625     0.2857143 ]
 [0.072      0.08333334 0.2857143 ]
 [0.138      0.10416666 0.2857143 ]
 [0.704      0.125      0.2857143 ]
 [0.539      0.14583333 0.2857143 ]
 [0.698      0.16666667 0.2857143 ]
 [0.757      0.1875     0.2857143 ]
 [0.671      0.20833333 0.2857143 ]
 [0.683      0.22916667 0.2857143 ]
 [0.676      0.25       0.2857143 ]
 [0.553      0.27083334 0.2857143 ]
 [0.757      0.29166666 0.2857143 ]
 [0.768      0.3125     0.2857143 ]
 [0.588      0.33333334 0

In [15]:
from sklearn.metrics.pairwise import rbf_kernel
import pickle

# note 这一个cell的目的，是构建cer-e的邻接矩阵

# note 首先加载数据
count_nodes = 485
count_timestamps = 25728
dataset_dir = "/data3/hanzhi/BasicTS/datasets/CER-E"
data = np.memmap(os.path.join(dataset_dir, "data.dat"), mode="r", shape=(count_timestamps, count_nodes), dtype=np.float32)
print(data.shape)
# note 然后计算每一周的任意两个时间序列的相似性
period = 48 * 7
train_len = int(count_timestamps * 0.7)
data = data[:train_len]
# 计算标准化
sim = np.zeros((data.shape[1], data.shape[1]))
tot = 0.
for i in tqdm(range(period, len(data), period)):
    # (nodes, period)
    xi = data[i - period:i].T
    # (period, period)
    si = np.corrcoef(xi)
    sim += si
    tot += 1.
sim /= tot
i = np.arange(count_nodes)
sim[i, i] = 0.
print(np.mean(sim), np.max(sim), np.min(sim))

# note 最后对每一个时间序列，计算与这条时间序列最相似的10条，对于时间序列i，让这十条时间序列adj[i][j]=1，默认情况下，不指向自己
# 到时候GWN会自己把它反过来
k = 9
indices = np.argpartition(sim, -k, axis=1)[:, -k:]
# 创建全零的mask数组，并将对应位置标记为True
mask = np.zeros_like(sim, dtype=np.float32)
np.put_along_axis(mask, indices, True, axis=1)
print(np.sum(mask))
# 应用mask
masked_arr = sim * mask
print(np.mean(masked_arr), np.max(masked_arr), np.min(masked_arr))
pickle.dump(mask, open(os.path.join(dataset_dir, "adj_correlation.pkl"), "wb"))

(25728, 485)


100%|██████████| 53/53 [00:00<00:00, 222.31it/s]

0.05183529976004236 0.998338713046769 -0.6031537180581951
4365.0
0.014066909453355208 0.998338713046769 0.0
